In [5]:
import pandas as pd
import os
import numpy as np
import collections
from tqdm import tqdm

In [6]:
variantDF1= pd.read_csv("/Users/loftum/Desktop/LeeLab/LeonardoDaVinci/JCVI/ISOGG_SNP_Index_260821.csv")
goodRows=[]
for row in variantDF1.index:
    if '.' in str(variantDF1.at[row,'Build 38 #']) or 'del' in str(variantDF1.at[row,'Mutation info']):
        continue
    else:
        goodRows.append(row)


variantDF11 = variantDF1.loc[goodRows].copy()
markerDict1 = collections.Counter(variantDF11['Name'])
print(len(markerDict1))

markerDict2 = {x:y for x,y in markerDict1.items() if y ==1}
print(len(markerDict2))

goodMarkers=[]
for row in variantDF11.index:
    if variantDF11.at[row,'Name'] in markerDict2:
        goodMarkers.append(row)

variantDF = variantDF11.loc[goodMarkers].copy()

91255
90002


In [9]:
import re
positionDict={}
for position in tqdm(set(variantDF['Build 38 #'])):
    tempVariantDF = variantDF[variantDF['Build 38 #']==position].copy()
    try:
        positionDict[position]={'haploGroups':list(set(tempVariantDF['Haplogroup'])),'MarkerID':list(tempVariantDF['Name'])[0], 'Ancestral':list(tempVariantDF['Mutation info'])[0].split("->")[0], 'Derived':list(tempVariantDF['Mutation info'])[0].split("->")[1]}
    except:
        continue

print(len(positionDict))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72641/72641 [02:27<00:00, 492.48it/s]

72637


In [10]:
sampleDictionary = {}
directory = '/Users/loftum/Desktop/LeeLab/LeonardoDaVinci/JCVI/variants/finalVCFs_Map5/'

for filename in os.listdir(directory):
    if filename == ".DS_Store":
        continue

    print(filename)
    sampleDictionary[filename] = {}

    tempList = []
    path = os.path.join(directory, filename)

    with open(path, 'r') as f:
        for line in f:
            if line.startswith("##"):
                continue
            tempList.append(line.split())

    tempDF = pd.DataFrame(tempList)
    tempDF.columns = tempDF.iloc[0]
    tempDF = tempDF.iloc[1:].reset_index(drop=True)

    tempDF1 = tempDF[~tempDF['INFO'].str.contains("INDEL", na=False)].copy()

    print(len(tempDF1))

    for row in tempDF1.index:
        
        fileColumn = tempDF1.columns[-1]
        position = str(tempDF1.at[row,'POS'])
        ref = str(tempDF1.at[row,'REF'])
        alt = str(tempDF1.at[row,'ALT'])
        info = str(tempDF1.at[row,'INFO'])

        if "AD=" not in info:
            continue

        #This will give me either just a number (1) or two numbers ('0,1')
        AD = info.split("AD=")[1].split(";")[0]

        # reference only
        if alt == ".":
            #if its invariant than the reference gets the reads
            refCounts = int(AD)
            altCounts = 0

        #else if there is a variant the ref and alt get their respective reads
        else:
            refCounts, altCounts = map(int, AD.split(","))

        #If this position is not informative then skip otherwise continue
        if position not in positionDict:
            continue



        if refCounts ==0 and altCounts ==0:
            continue
            
        else:
            #for each of the haplogroups that has this marker ID informative position
            for group in positionDict[position]['haploGroups']:
    
    
                #The MarkerID info
                ancestralSNP = positionDict[position]['Ancestral']
                derivedSNP = positionDict[position]['Derived']
                marker = positionDict[position]['MarkerID']
                
                #if the group does not exist in our sample dictionary then make an entry for the haplogroup
                #Keep track of how many ancestral or derived hits this group has and the different positions linked to the haplogroup
                if group not in sampleDictionary[filename]:
                    sampleDictionary[filename][group] = {"Ancestral": 0, "Derived": 0, 'Positions':{}}
                else:
                    pass
        
                #If the alt was a dot that means this sample carries that reference (invariant)
                #Now check for what the reference has
                if alt == '.':
                    if ref == ancestralSNP:
                        sampleDictionary[filename][group]['Positions'][position]={'MarkerID':marker,'ReferenceCounts':refCounts, 'SampleCounts':0,'AncestralSNP':ancestralSNP, 'DerivedSNP':derivedSNP, 'SampleContains':ref, 'FLAG':'invariant'}
                        sampleDictionary[filename][group]["Ancestral"] += refCounts
                    elif ref == derivedSNP:
                        sampleDictionary[filename][group]['Positions'][position]={'MarkerID':marker,'ReferenceCounts':0, 'SampleCounts':refCounts,'AncestralSNP':ancestralSNP, 'DerivedSNP':derivedSNP,'SampleContains':ref,'FLAG':'invariant'}
                        sampleDictionary[filename][group]["Derived"] += refCounts
                    else:
                        print('ODDITY', filename, group, position, ref, alt, ancestralSNP, derivedSNP)
                else:
    
                    if alt == ancestralSNP:
                        sampleDictionary[filename][group]['Positions'][position]={'MarkerID':marker,'ReferenceCounts':refCounts, 'SampleCounts':altCounts,'AncestralSNP':ancestralSNP, 'DerivedSNP':derivedSNP, 'ReferenceContains':ref, 'SampleContains':alt,'FLAG':'variant'}
                        sampleDictionary[filename][group]["Ancestral"] += altCounts
                        
                    elif alt == derivedSNP:
                        sampleDictionary[filename][group]['Positions'][position]={'MarkerID':marker,'ReferenceCounts':refCounts, 'SampleCounts':altCounts,'AncestralSNP':ancestralSNP, 'DerivedSNP':derivedSNP,'ReferenceContains':ref,'SampleContains':alt,'FLAG':'variant'}
                        sampleDictionary[filename][group]["Derived"] += altCounts
                        
                    else:
                        print('ODDITY', filename, group, position, ref, alt, ancestralSNP, derivedSNP)

SR-0023.map5.sg5.vcf
12444
SR-0024.map5.sg5.vcf
13055
SR-0012.map5.sg5.vcf
180
SR-0015.map5.sg5.vcf
2882666
WTF SR-0015.map5.sg5.vcf I2a1b1 6892475 T . TC TCC
WTF SR-0015.map5.sg5.vcf J1a2a1a2c~ 8205135 G . J A
WTF SR-0015.map5.sg5.vcf J1a2a1a2d2b2b2c2~ 11865699 G . C T
WTF SR-0015.map5.sg5.vcf D1a2a2b~ 12059415 C . T A
WTF SR-0015.map5.sg5.vcf H3b1 16818281 G T G C
WTF SR-0015.map5.sg5.vcf I2a2b1e1~ 17356300 C A C T
WTF SR-0015.map5.sg5.vcf I2a1b1a2b1a1a2c~ 19497475 C A C T
WTF SR-0015.map5.sg5.vcf C 19648238 C . G A
WTF SR-0015.map5.sg5.vcf G2a2b2a1a1b1a1b1a3~ 21212500 A . C T
11422-0008.map5.sg5.vcf
167
5540-011.map5.sg5.vcf
2093
SR-0007.map5.sg5.vcf
1732
SR-0025.map5.sg5.vcf
867227
WTF SR-0025.map5.sg5.vcf J1a2a1a2d2b2b2c2~ 11865699 G . C T
WTF SR-0025.map5.sg5.vcf C2a1a2a2b1~ 12122652 T C T A
WTF SR-0025.map5.sg5.vcf J2b2a1a1b1a~ 16123287 A G A C
SR-0022.map5.sg5.vcf
389986
WTF SR-0022.map5.sg5.vcf H2a1~ 11865554 C A C G
SR-0006.map5.sg5.vcf
351
SR-0001.map5.sg5.vcf
99493
SR-0014.

In [14]:
for sampleKey in sampleDictionary.keys():
    with open('/Users/loftum/Desktop/LeeLab/LeonardoDaVinci/JCVI/haplogroup_dataframes_map5/'+str(sampleKey)+'.fullInformation.ISOGG.csv', 'a+') as file:
        file.write("Sample\tMarkerID\thgID\tcoord\tAncAllele\tDerAllele\tSampleAllele\tAnceAlleleDP\tDerAlleleDP\tVarianceFlag\n")
        usedHaplogroups = {}
    
        for row in variantDF.index:

            #Unique haplogroup ID
            hg = variantDF.at[row,'Haplogroup']

            #If we already have done this haplogroup no need to do again
            if hg in usedHaplogroups:
                continue
    
            # mark as used now (prevents duplicated work)
            usedHaplogroups[hg] = 1  
    
            if hg not in sampleDictionary[sampleKey]:
                continue
    
            for pos, info in sampleDictionary[sampleKey][hg]['Positions'].items():
                if str(info['SampleContains']) == str(info['AncestralSNP']):

                    if str(info['FLAG']) == 'invariant':
                        file.write(
                            str(sampleKey)+ "\t"
                            + str(info['MarkerID']) + "\t"
                            + str(hg) + "\t"
                            + str(pos) + "\t"
                            + str(info['AncestralSNP']) + "\t"
                            + str(info['DerivedSNP']) + "\t"
                            + str(info['SampleContains']) + "\t"
                            + str(info['ReferenceCounts']) + "\t"
                            + str(info['SampleCounts']) + "\t"
                            + str(info['FLAG']) + "\n"
                        )
                    else:
                        file.write(
                            str(sampleKey)+ "\t"
                            + str(info['MarkerID']) + "\t"
                            + str(hg) + "\t"
                            + str(pos) + "\t"
                            + str(info['AncestralSNP']) + "\t"
                            + str(info['DerivedSNP']) + "\t"
                            + str(info['SampleContains']) + "\t"
                            + str(info['SampleCounts']) + "\t"
                            + str(info['ReferenceCounts']) + "\t"
                            + str(info['FLAG']) + "\n"
                        )
                        
                else:
                    file.write(
                            str(sampleKey)+ "\t"
                            + str(info['MarkerID']) + "\t"
                            + str(hg) + "\t"
                            + str(pos) + "\t"
                            + str(info['AncestralSNP']) + "\t"
                            + str(info['DerivedSNP']) + "\t"
                            + str(info['SampleContains']) + "\t"
                            + str(info['ReferenceCounts']) + "\t"
                            + str(info['SampleCounts']) + "\t"
                            + str(info['FLAG']) + "\n"
                        )
                        

In [15]:
for sampleKey in sampleDictionary.keys():
    with open('/Users/loftum/Desktop/LeeLab/LeonardoDaVinci/JCVI/haplogroupSummaries_map5/'+str(sampleKey)+'.haploLevelInfo.ISOGG.csv', 'a+') as file:
        file.write("Sample\thgID\tAnceAlleleDP\tDerAlleleDP\n")
        usedHaplogroups = {}
    
        for row in variantDF.index:

            #Unique haplogroup ID
            hg = variantDF.at[row,'Haplogroup']

            #If we already have done this haplogroup no need to do again
            if hg in usedHaplogroups:
                continue
    
            # mark as used now (prevents duplicated work)
            usedHaplogroups[hg] = 1  
    
            if hg not in sampleDictionary[sampleKey]:
                continue

            AncestralCount = sampleDictionary[sampleKey][hg]['Ancestral']
            DerivedCount = sampleDictionary[sampleKey][hg]['Derived']

            file.write(sampleKey + "\t" + hg +"\t" + str(AncestralCount) +'\t' + str(DerivedCount) +'\n')